### init

In [13]:
%load_ext autoreload
import matplotlib
%reload_ext autoreload
%autoreload 2

#%matplotlib notebook
#%matplotlib widget

import warnings
warnings.simplefilter("ignore")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [14]:
import logging
import signal, threading, os, time
import logging

from IPython.core.debugger import set_trace
from IPython.terminal.embed import embed

import os, sys


import ccxt
import pandas as pd

logger = logging.getLogger('notebook')
logger.setLevel(logging.INFO)

In [15]:
import mplfinance as mpf
#matplotlib.use('Agg')
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [8, 12]


In [16]:

try:
    from settings import HTTP_PROXY
except ImportError:
    HTTP_PROXY = None
    
# (!!!)You should change this according to your network environment
if HTTP_PROXY is None:
    HTTP_PROXY = 'http://127.0.0.1:1087'    

def kline_to_df(arr) -> pd.DataFrame:
    kline = pd.DataFrame(
        arr,
        columns=['ts', 'open', 'high', 'low', 'close', 'volume' ])
    kline.index = pd.to_datetime(kline.ts, unit='ms')
    kline.drop('ts', axis=1, inplace=True)
    return kline
    

In [17]:

PROXIES = {
    'http': HTTP_PROXY,
    'https': HTTP_PROXY,
}

ccxt_options = {'proxies': PROXIES}

ok = 'okex'
bn = 'binance'

client_list = [bn, ok]

## Step by Step

In [19]:
# prepare client
binance = getattr(ccxt, 'binance')(ccxt_options)
markerts = binance.load_markets()
symbol = 'BTC/USDT'
period = '4h'

In [20]:
# fetch raw kline data via ccxt unified api
kline_data = binance.fetch_ohlcv(symbol, period, limit=1000)

kline = kline_to_df(kline_data)

In [21]:
print('raw ccxt kline data:\n', kline_data[:4])

raw ccxt kline data:
 [[1658736000000, 21962.33, 22110.98, 21868.36, 21941.25, 22048.18138], [1658750400000, 21942.18, 22003.12, 21683.4, 21895.79, 32159.10297], [1658764800000, 21893.94, 22021.47, 21552.53, 21890.17, 27200.4675], [1658779200000, 21892.66, 22259.98, 21250.0, 21310.9, 43229.18467]]


In [22]:
print('convert to dataframe:\n', kline[:4])

convert to dataframe:
                          open      high       low     close       volume
ts                                                                      
2022-07-25 08:00:00  21962.33  22110.98  21868.36  21941.25  22048.18138
2022-07-25 12:00:00  21942.18  22003.12  21683.40  21895.79  32159.10297
2022-07-25 16:00:00  21893.94  22021.47  21552.53  21890.17  27200.46750
2022-07-25 20:00:00  21892.66  22259.98  21250.00  21310.90  43229.18467


In [23]:
patterns, predict_patterns = detector.search_patterns(kline, only_last=False, last_n=4, plot=False, predict=True)

In [24]:
for pat in patterns:
    msg = f'{symbol} {period} \npatterns found: {pat[1]}, {pat[0]}, \n {pat[2]}, {pat[3]}'
    print(msg)


for pat in predict_patterns:
    msg = '\n'.join([f'{p} {v}' for p,v in list(zip([str(dt) for dt in pat[1]], [p for p in pat[0]]))])
    msg = f'{symbol} {period} {msg} {pat[2]} {pat[3]}'
    print(msg)

BTC/USDT 4h 
patterns found: [Timestamp('2022-08-17 04:00:00'), Timestamp('2022-08-20 20:00:00'), Timestamp('2022-08-21 20:00:00'), Timestamp('2022-08-23 04:00:00'), Timestamp('2022-08-24 16:00:00')], [['H', 24446.71, 137], ['L', 20761.9, 159], ['H', 21800.0, 165], ['L', 20890.14, 173], ['H', 21900.0, 182]], 
 bearish abcd, {'AB': 0.8764666217127463, 'BC': 1.1099070186622118, 'AB=CD': 0.9727964550621349}
BTC/USDT 4h 
patterns found: [Timestamp('2022-08-15 00:00:00'), Timestamp('2022-08-20 20:00:00'), Timestamp('2022-08-21 20:00:00'), Timestamp('2022-08-23 04:00:00'), Timestamp('2022-08-24 16:00:00')], [['H', 25211.32, 124], ['L', 20761.9, 159], ['H', 21800.0, 165], ['L', 20890.14, 173], ['H', 21900.0, 182]], 
 bearish abcd, {'AB': 0.8764666217127463, 'BC': 1.1099070186622118, 'AB=CD': 0.9727964550621349}
BTC/USDT 4h 
patterns found: [Timestamp('2022-10-04 20:00:00'), Timestamp('2022-10-13 12:00:00'), Timestamp('2022-10-14 00:00:00'), Timestamp('2022-10-21 12:00:00'), Timestamp('2022-10

###  Parameters

In [18]:
from harmonic_functions import HarmonicDetector
#from HarmonicPatterns.harmonic import send_alert, search_function
from functools import partial
import time
from multiprocessing import Pool

PERIODS = ['1h', '4h', '1d']

################
# HarmonicDetector params
# err_allowed: The error rate that detector would allow, usually 0.05 < err_rate < 0.1
# strict: for example 0.618 * (1 - err_rate) < pattern < 0.618 * (1 + err_rate), the base should be a fibbonacci number, like 0.382, 0.618, 0.786, 1.618....
# predict_err_rate: # similiar to err_allowed but used in predict


################
# search params
# only_last: find patterns in history or not
# symbols: symbols list to find
# periods: periods that used in ccxt
# last_n: if only_last is set, how near should the bar be
# alert: send alerts when patterns found
# plot: plot patterns when patterns found
def search_function(detector, exchange_id, symbols, periods=PERIODS, ccxt_args={}, savefig=False, predict=True, only_last=False, alert=False, plot=True):
    client = getattr(ccxt, exchange_id)(ccxt_args)
    client.load_markets()
    RETRY_TIMES=3
    for symbol in symbols:
        for period in periods:
            print(f'------------------calculating {symbol} {period}------------------')
            retry = RETRY_TIMES
            while retry>0:
                try:
                    df = kline_to_df(client.fetch_ohlcv(symbol, period, limit=1000))

                    patterns, predict_patterns = detector.search_patterns(df, only_last=only_last, last_n=4, plot=plot, predict=predict)
                    break
                except Exception as e:
                    print(e)
                    retry -= 1
                    if retry==0: raise
                    continue
            for pat in patterns:
                msg = f'{symbol} {period} \npatterns found: {pat[1]}, {pat[0]}, \n {pat[2]}, {pat[3]}'
                print(msg)
                if alert and pat[0][-1][2] == len(df)-1:
                    send_alert(f'Pattern_Found_{symbol}_{period}', msg)

            for pat in predict_patterns:
                msg = '\n'.join([f'{p} {v}' for p,v in list(zip([str(dt) for dt in pat[1]], [p for p in pat[0]]))])
                msg = f'{symbol} {period} {msg} {pat[2]} {pat[3]}'
                print(msg)
                if alert:
                    send_alert(f'Pattern_Predict_{symbol}_{period}', msg)


detector = HarmonicDetector(error_allowed=0.07, strict=True, predict_err_rate=0.07)
search = partial(search_function, detector, ccxt_args=ccxt_options, only_last=False, plot=True)

#PERIODS = ['5m','15m', '30m']


###  DETECT BTC

In [ ]:
s = ['BTC/USDT', 'ETH/USDT']
search('binance', s, periods = ['1h', '4h'])

###  BTC USDT

In [ ]:
s = ['BTC/USDT']

search(ok, s, periods=['1h', '4h'], predict=True, only_last=False, alert=False, plot=True)
